In [1003]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy import hstack

In [1088]:
data1 = pd.read_excel("C:/Users/samba/OneDrive/Desktop/TAMU CORPUS/Driscoll/combined_datasets/average_ch_5.xlsx")
data1

,day,average_ch_20_5,average_ch_21_5
0,0,0.000000,0.000000
1,13,0.014270,0.009240
2,25,0.082395,0.007381
3,35,0.154837,0.007807
4,41,0.185905,0.037288
5,49,0.227330,0.079972
6,55,0.395063,0.153447
7,62,0.459235,0.169336
8,69,0.523408,0.179791
9,77,0.653634,0.186437


In [1089]:
#in_seq1 = data1['average_eg_19_5'].to_numpy()
in_seq2 = data1['average_ch_20_5'].to_numpy()
out_seq = data1['average_ch_21_5'].to_numpy()
#in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
dataset = np.hstack((in_seq2, out_seq))
print(dataset)

[[0.         0.        ]
 [0.01426998 0.00923982]
 [0.0823946  0.00738051]
 [0.15483664 0.00780737]
 [0.18590514 0.03728786]
 [0.22732983 0.07997232]
 [0.39506259 0.15344746]
 [0.45923516 0.16933618]
 [0.52340774 0.17979087]
 [0.65363373 0.18643669]
 [0.72601237 0.15074486]
 [0.79839101 0.17394751]
 [0.86042984 0.22374813]
 [0.81050052 0.24419165]
 [0.79       0.35994284]]


In [1090]:
data2 = dataset

In [1097]:
training_set = data2[0:13]
testing_set = data2[10:13]

In [1098]:
training_set

array([[0.        , 0.        ],
       [0.01426998, 0.00923982],
       [0.0823946 , 0.00738051],
       [0.15483664, 0.00780737],
       [0.18590514, 0.03728786],
       [0.22732983, 0.07997232],
       [0.39506259, 0.15344746],
       [0.45923516, 0.16933618],
       [0.52340774, 0.17979087],
       [0.65363373, 0.18643669],
       [0.72601237, 0.15074486],
       [0.79839101, 0.17394751],
       [0.86042984, 0.22374813]])

In [1099]:
x_input = testing_set[:,:-1]
x_input

array([[0.72601237],
       [0.79839101],
       [0.86042984]])

In [1100]:
#vanilla lstm
# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X,y = list(), list()
for i in range(len(training_set)):
    #find the end of this pattern
    end_ix = i + n_steps_in
    out_end_ix = end_ix + n_steps_out-1
    #check if we are beyond the sequence
    if out_end_ix > len(training_set):
        break
    #gather input and output parts of the pattern
    seq_x, seq_y = training_set[i:end_ix, :-1], training_set[end_ix-1:out_end_ix, -1]
    X.append(seq_x)
    y.append(seq_y)
X = np.array(X)
y = np.array(y)
print (X)
print (y)

[[[0.        ]
  [0.01426998]
  [0.0823946 ]]

 [[0.01426998]
  [0.0823946 ]
  [0.15483664]]

 [[0.0823946 ]
  [0.15483664]
  [0.18590514]]

 [[0.15483664]
  [0.18590514]
  [0.22732983]]

 [[0.18590514]
  [0.22732983]
  [0.39506259]]

 [[0.22732983]
  [0.39506259]
  [0.45923516]]

 [[0.39506259]
  [0.45923516]
  [0.52340774]]

 [[0.45923516]
  [0.52340774]
  [0.65363373]]

 [[0.52340774]
  [0.65363373]
  [0.72601237]]

 [[0.65363373]
  [0.72601237]
  [0.79839101]]]
[[0.00738051 0.00780737]
 [0.00780737 0.03728786]
 [0.03728786 0.07997232]
 [0.07997232 0.15344746]
 [0.15344746 0.16933618]
 [0.16933618 0.17979087]
 [0.17979087 0.18643669]
 [0.18643669 0.15074486]
 [0.15074486 0.17394751]
 [0.17394751 0.22374813]]


In [1101]:
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [1102]:
# demonstrate prediction
x_input = testing_set[:,:-1]
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[0.19701375 0.20488545]]
